### 1. 서울 열린데이터 광장에서 수집한 서울시 인구수 데이터셋 가공

In [2]:
import pandas as pd

s_pop = pd.read_csv('Seoul_population.txt', sep = "\t", encoding="utf-8")
s_pop

,기간,자치구,세대,인구,인구.1,인구.2,인구.3,인구.4,인구.5,인구.6,인구.7,인구.8,인구밀도,인구밀도.1,세대당인구,65세이상고령자
0,기간,자치구,세대,합계,합계,합계,한국인,한국인,한국인,등록외국인,등록외국인,등록외국인,인구밀도,인구밀도,세대당인구,65세이상고령자
1,기간,자치구,세대,계,남자,여자,계,남자,여자,계,남자,여자,인구밀도(명/㎢),면적(㎢),세대당인구,65세이상고령자
2,2007,합계,"4,046,086","10,421,782","5,174,655","5,247,127","10,192,710","5,063,188","5,129,522","229,072","111,467","117,605","17,219",605.25,2.52,"852,235"
3,2007,종로구,"71,968","173,843","87,001","86,842","165,846","83,104","82,742","7,997","3,897","4,100","7,271",23.91,2.42,"19,127"
4,2007,중구,"57,473","137,435","68,516","68,919","130,044","64,908","65,136","7,391","3,608","3,783","13,799",9.96,2.39,"14,907"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,2019,관악구,"268,559","517,334","259,194","258,140","500,094","251,009","249,085","17,240","8,185","9,055",-,-,1.86,"75,194"
336,2019,서초구,"173,199","435,107","208,181","226,926","430,826","206,039","224,787","4,281","2,142","2,139",-,-,2.49,"57,136"
337,2019,강남구,"232,981","550,209","263,345","286,864","545,169","260,806","284,363","5,040","2,539","2,501",-,-,2.34,"71,025"
338,2019,송파구,"278,711","682,741","330,412","352,329","675,961","327,164","348,797","6,780","3,248","3,532",-,-,2.43,"87,334"


In [2]:
# 불필요한 index 제거
s_pop.drop([0, 1], inplace=True)
s_pop.reset_index(inplace=True, drop=True)

drop_idx = s_pop[s_pop['자치구'] == '합계'].index
s_pop.drop(drop_idx, inplace=True)
s_pop.reset_index(inplace=True, drop=True)

# 불필요한 column 제거
s_pop.drop(['세대', '인구밀도', '인구밀도.1', '인구.1', '인구.2', '인구.3', '인구.4', '인구.5', '인구.6', '인구.7', '인구.8', '세대당인구', '65세이상고령자'],
           axis=1, inplace=True)

# column명 변경
s_pop.rename(columns={'기간':'연도'}, inplace=True)

# 인구 Column의 data형 변환
s_pop['인구'] = s_pop['인구'].str.replace(',', '').astype(int)

# 연도, 자치구 순서로 오름차순 정렬
s_pop.sort_values(by=['연도', '자치구'], inplace=True)

In [3]:
s_pop.to_csv('연도별 서울 자치구 인구수.csv', encoding='cp949', index=False)

del s_pop # 메모리 제거

In [3]:
import pandas as pd
import numpy as np

# Load Data
Seoul_0719 = pd.read_csv('서울시 개별사고정보(2007_2019).csv',  encoding="cp949")
Seoul_pop = pd.read_csv('연도별 서울 자치구 인구수.csv',  encoding="cp949")

# 발생일로부터 발생연도와 발생월을 int형으로 추출
Seoul_0719['발생일'] = Seoul_0719['발생일'].astype(str)
Seoul_0719['발생연도'] = Seoul_0719['발생일'].str.slice(start=0, stop=4).astype(int)
Seoul_0719['발생월'] = Seoul_0719['발생일'].str.slice(start=4, stop=6).astype(int)

In [123]:
# 새로운 DataFrame 생성 전, 발생 건수를 list로 추출
case_count = []
year_local_count = pd.DataFrame(Seoul_0719.groupby(['발생연도', '발생지_시군구']).size(), columns=['발생건수'])

for i in range(13):
    for j in range(25):
        case_count.append(int(year_local_count['발생건수'][2007+i][j]))

In [124]:
# 발생건수 DataFrame 생성
ser_dict = {'발생건수':case_count}
df_tmp = pd.DataFrame(ser_dict)

# DataFrame 병합
Seoul_pop = Seoul_pop.join(df_tmp)

# 인구비율에 따른 발생건수를 계산한 Column 추가
Seoul_pop['인구 수 대비 사고비율'] = Seoul_pop['발생건수'] / Seoul_pop['인구'] * 100

# csv로 저장
Seoul_pop.to_csv('연도별 서울 자치구 인구수와 사건발생 건수.csv', encoding='cp949', index=False)

In [9]:
# folium 설치
!pip install folium

In [133]:
import json
import folium
import warnings
warnings.simplefilter(action = "ignore", category = FutureWarning)

geo_path = 'skorea_municipalities_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))

## 1) 시각화 이미지 확인(연도/항목 선택형)

In [184]:
def draw_map(df, item):
    geo = folium.Map(location=[37.5502, 126.982], zoom_start=11) #tiles='Stamen Toner'

    geo.choropleth(geo_data=geo_str,
                   data=df[item],
                   columns=(df.index, df[item]),
                   #columns=(Seoul_pop[Seoul_pop['연도'] == 2007]['자치구'], Seoul_pop[Seoul_pop['연도'] == 2007]['발생건수']),
                   fill_color='PuRd', #PuRd, YlGnBu
                   key_on='feature.id')
    display(geo)

    
def main():
    print("1. 자치구별 교통사고 발생건수")
    print("2. 자치구별 인구 수 대비 사고발생 비율")
    sel_item = int(input("확인하고자하는 항목의 번호를 입력하시오: "))
    sel_year = int(input("2007 ~ 2019년 사이에서 확인할 연도를 입력하시오: "))
    
    sel_item_dict = {1:'발생건수', 2:'인구 수 대비 사고비율'}
    
    tmp_df = Seoul_pop[Seoul_pop['연도'] == sel_year][['자치구', sel_item_dict[sel_item]]].set_index('자치구')
    
    draw_map(tmp_df, sel_item_dict[sel_item])
    
if __name__ == '__main__':
     main()

1. 자치구별 교통사고 발생건수
2. 자치구별 인구 수 대비 사고발생 비율
확인하고자하는 항목의 번호를 입력하시오: 1
2007 ~ 2019년 사이에서 확인할 연도를 입력하시오: 2007


## 2) 시각화 이미지 확인(2007년부터 2019년 시간 순으로 이미지 변경)

In [185]:
from IPython.display import display, clear_output
import time

def draw_map(df, year, item):
    geo = folium.Map(location=[37.5502, 126.982], zoom_start=11) #tiles='Stamen Toner'

    geo.choropleth(geo_data=geo_str,
                   data=df[item],
                   columns=(df.index, df[item]),
                   fill_color='PuRd', #PuRd, YlGnBu
                   legend_name='%d년'%year,
                   key_on='feature.id')
    display(geo)
    
def main():
    print("1. 자치구별 교통사고 발생건수")
    print("2. 자치구별 인구 수 대비 사고발생 비율")
    sel_item = int(input("확인하고자하는 항목의 번호를 입력하시오: "))
    
    sel_item_dict = {1:'발생건수', 2:'인구 수 대비 사고비율'}
    
    # 2007년부터 1초 간격으로 2019년까지 지도에 시각화한 이미지를 보여줌
    for i in range(13):
        clear_output(wait=True)
        year = 2007 + i
        tmp_df = Seoul_pop[Seoul_pop['연도'] == year][['자치구', sel_item_dict[sel_item]]].set_index('자치구')
        draw_map(tmp_df, year, sel_item_dict[sel_item])
        time.sleep(1)
    
if __name__ == '__main__':
     main()